In [18]:
import kaleido

from dash import Dash, dcc, html, Input, Output, ctx
from pathlib import Path
from dash.exceptions import PreventUpdate

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots as subplots

import pandas as pd


import numpy as np

In [2]:
ALL_CAUSES = 'Waste - agri-food systems'

DEFAULT_YEAR = 2019
DEFAULT_COUTRY = 'France'
SIZE_MAX = 40

In [3]:
df = pd.read_csv('dasy.csv')

In [5]:
# ============================================================
# Ratio of food-waste related emissions over total emissions
# ============================================================
# Default map showing the ratio of food-waste related emissions over total emissions
dff = df[(df.cause == ALL_CAUSES) & (df.year == DEFAULT_YEAR)]
default_fig_map_feote = px.scatter_geo(dff, locations="iso_alpha", 
                           color="value_over_emission", 
                           size="value_over_emission", 
                           hover_name="country",
                           size_max=20, 
                           color_continuous_scale=px.colors.sequential.Viridis_r,
                        #    animation_frame="year",
                           projection="natural earth")

default_fig_map_feote.update_layout(coloraxis_colorbar=dict(
            title="Ratio food-waste / total",
            yanchor="top",
            # y=0.99,
            # xanchor="left",
            # x=0.01,
            orientation='h',
            len=1,
         ),
         # title_text='Food-waste emissions over total emissions',
      )
# default_fig_map_feote.update_layout(showlegend=False)
default_fig_map_feote.update(layout_coloraxis_showscale=False)

default_fig_map_feote.write_image("feote.svg")

In [7]:
# ============================================================
# Food related emissions
# ============================================================
df_f = df[(df.cause == 'Waste - agri-food systems') & (df.year == DEFAULT_YEAR)]
df_f.sort_values(by=['year'], inplace=True)

default_fig_map_fre = px.scatter_geo(df_f, locations="iso_alpha", 
                           color="value", 
                           size="value", 
                           hover_name="country",
                           # symbol='cause',
                           size_max=20,
                           color_continuous_scale=px.colors.sequential.Greys,
                        #    animation_frame="year",
                           projection="natural earth") 
default_fig_map_fre.update(layout_coloraxis_showscale=False)

default_fig_map_fre.write_image("fre.svg")

/tmp/ipykernel_9257/4155856614.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
# ============================================================
# Different causes of food-waste related emissions
# ============================================================

# Default bar chart showing the difference causes of emissions
dff = df[df.cause != ALL_CAUSES]
dom_wastewater = dff[dff.cause == 'Domestic wastewater'].value.sum()
ind_wastewater = dff[dff.cause == 'Industrial wastewater'].value.sum()
sol_food_waste = dff[dff.cause == 'Solid food waste'].value.sum()

# print(dom_wastewater, ind_wastewater, sol_food_waste)
default_fig_bar_causes = px.bar(x=['Domestic wastewater', 'Industrial wastewater', 'Solid food waste'], 
        y=[dom_wastewater, ind_wastewater, sol_food_waste],
        color=['Domestic wastewater', 'Industrial wastewater', 'Solid food waste'])

default_fig_bar_causes.update_layout(
    title=f'Causes of emissions summed up since 1990',
    xaxis_title="Food-waste cause",
    yaxis_title="Emissions (kt)",
    legend_title="Food-waste cause",
)
default_fig_bar_causes.show()
default_fig_bar_causes.write_image("bar_causes.svg")

In [16]:
# ==========================================================
# LINE CHART - TREND OF FOOD-WASTE RELATED 
# EMISSIONS IN ALL COUNTRIES
# ==========================================================
# Default line chart showing the trend of food-waste related emissions
dff = df[df.cause != ALL_CAUSES]
ind_wastewater = []
home_wastewater = []
solid_waste = []
for year in range(1990, 2020):
    tdf = dff[dff['year'] == year]
    ind_wastewater.append(tdf[tdf['cause'] == 'Industrial wastewater'].value.sum())
    home_wastewater.append(tdf[tdf['cause'] == 'Domestic wastewater'].value.sum())
    solid_waste.append(tdf[tdf['cause'] == 'Solid food waste'].value.sum())
res = dict()
res['Year'] = np.array(range(1990, 2020))
res['Industrial wastewater'] = np.array(ind_wastewater)
res['Domestic wastewater'] = np.array(home_wastewater)
res['Solid food waste'] = np.array(solid_waste)
rdf = pd.DataFrame(res)
default_fig_line_causes = px.line(rdf, x='Year', 
                            y=['Domestic wastewater', 'Industrial wastewater', 'Solid food waste'],
                            color_discrete_sequence=px.colors.qualitative.Dark24)

default_fig_line_causes.update_layout(
    title=f'Trend of food-waste related emissions from 1990 to 2019',
    xaxis_title="Year",
    yaxis_title="Emissions (kt)",
    legend_title="Food-waste cause",
)

default_fig_line_causes.write_image("line_causes.svg")

In [26]:
# =================================================================================
# Top polluters (countries) from 1990 to 2019, cause: 'Waste - agri-food systems'
# =================================================================================
dff = df[df.cause == 'Waste - agri-food systems']
# Put the countries as columns and the years as rows
dff = dff.pivot(columns='country', values='value', index='year')
# dff = dff[dff.index >= 2002]
dff.dropna(axis=1, inplace=True)
dff = dff.sum().sort_values(ascending=False).head(20)

fig_bar_top_polluters = px.bar(dff, x=dff.index, y=dff.values)
fig_bar_top_polluters.update_layout(
    title=f'Top polluters from 1990 to 2019',
    xaxis_title="Country",
    yaxis_title="Emissions (kt)",
)

# =================================================================================
# Top polluters (countries) from 1990 to 2019, cause: 'Waste - agri-food systems',
# normalized by population
# =================================================================================
dff = df[df.cause == 'Waste - agri-food systems']
# Put the countries as columns and the years as rows
dff = dff.pivot(columns='country', values='value_per_capita', index='year')
# dff = dff[dff.index >= 2002]
dff.dropna(axis=1, inplace=True)
dff = dff.sum().sort_values(ascending=False).head(20)

fig_bar_top_polluters_per_capita = px.bar(dff, x=dff.index, y=dff.values)
fig_bar_top_polluters_per_capita.update_layout(
    title=f'Top polluters per capita from 1990 to 2019',
    xaxis_title="Country",
    yaxis_title="Emissions (kt) per capita",
)

# =================================================================================
# Top polluters (countries) from 1990 to 2019, cause: 'Waste - agri-food systems',
# normalized by GDP
# =================================================================================
dff = df[df.cause == 'Waste - agri-food systems']
# Put the countries as columns and the years as rows
dff = dff.pivot(columns='country', values='value_per_gdp', index='year')
# dff = dff[dff.index >= 2002]
dff.dropna(axis=1, inplace=True)
dff = dff.rename(columns={'Central African Republic': 'C. African Rep.'})
dff = dff.sum().sort_values(ascending=False).head(20)

fig_bar_top_polluters_per_gdp = px.bar(dff, x=dff.index, y=dff.values)
fig_bar_top_polluters_per_gdp.update_layout(
    title=f'Top polluters per GDP from 1990 to 2019',
    xaxis_title="Country",
    yaxis_title="Emissions (kt) per $",
)

fig = subplots(rows=3, cols=1)

fig.add_trace(fig_bar_top_polluters['data'][0], 1, 1)
fig.add_trace(fig_bar_top_polluters_per_capita['data'][0], 2, 1)
fig.add_trace(fig_bar_top_polluters_per_gdp['data'][0], 3, 1)

fig.update_layout(
    title=f'Ranking over different metrics',
)

fig.update_layout(height=1000, width=1000)
fig.show()
fig.write_image("bar_top_polluters.svg")